[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/stable-diffusion-webui-colab/blob/main/nightly/stable_diffusion_v2_1_base_webui_colab.ipynb)

In [ ]:
%cd /content

%env TF_CPP_MIN_LOG_LEVEL=1

!apt -y update -qq
!wget http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
!apt install -qq libunwind8-dev
!dpkg -i *.deb
%env LD_PRELOAD=libtcmalloc.so
!rm *.deb

!apt -y install -qq aria2 libcairo2-dev pkg-config python3-dev
!pip install -q torch==2.0.0+cu118 torchvision==0.15.1+cu118 torchaudio==2.0.1+cu118 torchtext==0.15.1 torchdata==0.6.0 --extra-index-url https://download.pytorch.org/whl/cu118 -U
!pip install -q xformers==0.0.19 triton==2.0.0 -U

!git clone -b v2.4 https://github.com/camenduru/ui
!git clone https://huggingface.co/embed/negative /content/ui/embeddings/negative
!git clone https://huggingface.co/embed/lora /content/ui/models/Lora/positive
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d /content/ui/models/ESRGAN -o 4x-UltraSharp.pth
!git clone https://github.com/camenduru/sd-civitai-browser /content/ui/extensions/sd-civitai-browser
!git clone https://github.com/camenduru/control /content/ui/extensions/control
!git clone https://github.com/fkunn1326/openpose-editor /content/ui/extensions/openpose-editor
!git clone https://github.com/camenduru/tunnels /content/ui/extensions/tunnels
%cd /content/ui
!git reset --hard
!git -C /content/ui/repositories/stable-diffusion-stability-ai reset --hard

!aria2c --console-log-level=error -c -x 16 -k 1M -s 16 https://huggingface.co/ckpt/ControlNet/resolve/main/canny-sd21-safe.safetensors -d /content/ui/extensions/control/models -o canny-sd21-safe.safetensors
!aria2c --console-log-level=error -c -x 16 -k 1M -s 16 https://huggingface.co/ckpt/ControlNet/resolve/main/depth-sd21-safe.safetensors -d /content/ui/extensions/control/models -o depth-sd21-safe.safetensors
!aria2c --console-log-level=error -c -x 16 -k 1M -s 16 https://huggingface.co/ckpt/ControlNet/resolve/main/hed-sd21-safe.safetensors -d /content/ui/extensions/control/models -o hed-sd21-safe.safetensors
!aria2c --console-log-level=error -c -x 16 -k 1M -s 16 https://huggingface.co/ckpt/ControlNet/resolve/main/openpose-sd21-safe.safetensors -d /content/ui/extensions/control/models -o openpose-sd21-safe.safetensors
!aria2c --console-log-level=error -c -x 16 -k 1M -s 16 https://huggingface.co/ckpt/ControlNet/resolve/main/scribble-sd21-safe.safetensors -d /content/ui/extensions/control/models -o scribble-sd21-safe.safetensors
!aria2c --console-log-level=error -c -x 16 -k 1M -s 16 https://huggingface.co/ckpt/ControlNet/raw/main/cldm_v21.yaml -d /content/ui/extensions/control/models -o canny-sd21-safe.yaml
!aria2c --console-log-level=error -c -x 16 -k 1M -s 16 https://huggingface.co/ckpt/ControlNet/raw/main/cldm_v21.yaml -d /content/ui/extensions/control/models -o depth-sd21-safe.yaml
!aria2c --console-log-level=error -c -x 16 -k 1M -s 16 https://huggingface.co/ckpt/ControlNet/raw/main/cldm_v21.yaml -d /content/ui/extensions/control/models -o hed-sd21-safe.yaml
!aria2c --console-log-level=error -c -x 16 -k 1M -s 16 https://huggingface.co/ckpt/ControlNet/raw/main/cldm_v21.yaml -d /content/ui/extensions/control/models -o openpose-sd21-safe.yaml
!aria2c --console-log-level=error -c -x 16 -k 1M -s 16 https://huggingface.co/ckpt/ControlNet/raw/main/cldm_v21.yaml -d /content/ui/extensions/control/models -o scribble-sd21-safe.yaml

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.ckpt -d /content/ui/models/Stable-diffusion -o v2-1_512-ema-pruned.ckpt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/stable-diffusion-2-1-base/raw/main/v2-inference.yaml -d /content/ui/models/Stable-diffusion -o v2-1_512-ema-pruned.yaml

!sed -i -e '''/from modules import launch_utils/a\import os''' /content/ui/launch.py
!sed -i -e '''/        prepare_environment()/a\        os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /content/ui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /content/ui/launch.py
!sed -i -e 's/\["sd_model_checkpoint"\]/\["sd_model_checkpoint","sd_vae","CLIP_stop_at_last_layers"\]/g' /content/ui/modules/shared.py

!python launch.py --listen --xformers --enable-insecure-extension-access --theme dark --gradio-queue --multiple